In [133]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

# Reading the Dataset

In [134]:
df = pd.read_csv("rawdata.csv")
print(df.shape)

(1146, 20)


# Fix Map most used Map
###### We want to analyze the different combinations of brawler types. As the combinations depend on the map, we fix our data frame to th most used map.
- first check which map is used the most

In [135]:
pd.value_counts(df['map'])

Backyard Bowl     903
Post Haste        119
Warped Arena       25
Triple Dribble     22
Super Stadium      17
Beach Ball         16
Field Goal         11
Pinhole Punt        9
Well Cut            7
Pinball Dreams      6
Penalty Kick        4
Galaxy Arena        4
Center Field        2
Shoulder Bash       1
Name: map, dtype: int64

- copy initial data frame and remove all games that are played on other maps

In [136]:
df_fixed_map = df.copy()
df_fixed_map = df_fixed_map[df_fixed_map['map'] == 'Backyard Bowl'] 

# Define Brawler Type Lists
###### We need to define the type for each brawler. The information is obtained from the game itself.

In [137]:

sniper_brawlers = ['COLT','RICO','8-BIT','PIPER','BROCK','BEA','PENNY','MR. P','SPIKE']
thrower_brawlers = ['DYNAMIKE','TICK','BARLEY']
heavyweight_brawlers = ['DARRYL','EL PRIMO','ROSA','BULL','FRANK','JACKY','BIBI']
fighter_brawlers = ['CARL','EMZ','BO','TARA','JESSIE','NITA','SHELLY']
support_brawlers = ['POCO','PAM','GENE','MAX','SPROUT','SANDY','GALE',]
assassin_brawlers = ['MORTIS','CROW','LEON']


# Add Brawler Types to the DataFrame
### Function to Assign Brawler Type

In [138]:
def assign_brawler_type(brawler):
    if brawler in sniper_brawlers:
        return 'SNIPER'
    elif brawler in thrower_brawlers:
        return 'THROWER'
    elif brawler in heavyweight_brawlers:
        return 'HEAVYWEIGHT'
    elif brawler in fighter_brawlers:
        return 'FIGHTER'
    elif brawler in support_brawlers:
        return 'SUPPORTER'
    elif brawler in assassin_brawlers:
        return 'ASSASSIN'
    else:
        return ''
    

### Add Brawler Types to DataFrame

In [139]:
df_fixed_map['Brawler1Type'] = df_fixed_map['Brawler1Name'].apply(assign_brawler_type)
df_fixed_map['Brawler2Type'] = df_fixed_map['Brawler2Name'].apply(assign_brawler_type)
df_fixed_map['Brawler3Type'] = df_fixed_map['Brawler3Name'].apply(assign_brawler_type)
df_fixed_map['Brawler4Type'] = df_fixed_map['Brawler4Name'].apply(assign_brawler_type)
df_fixed_map['Brawler5Type'] = df_fixed_map['Brawler5Name'].apply(assign_brawler_type)
df_fixed_map['Brawler6Type'] = df_fixed_map['Brawler6Name'].apply(assign_brawler_type)


# Check if same Type is used by multiple Players
###### If used by multiple players we add _<<number>> to the type to be able to compute the frequent itemsets

In [140]:
counter = 0
for index, row in df_fixed_map.iterrows():
    if row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] != row['Brawler3Type']:
        df_fixed_map.loc[index, 'Brawler2Type'] = row['Brawler2Type']+'_2'
    elif row['Brawler1Type'] == row['Brawler2Type'] and row['Brawler1Type'] == row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler2Type'] = row['Brawler2Type']+'_2'
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_3'
    elif row['Brawler1Type'] == row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] != row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
    elif row['Brawler1Type'] != row['Brawler3Type'] and row['Brawler1Type'] != row['Brawler2Type'] and row['Brawler2Type'] == row['Brawler3Type']:
        df_fixed_map.loc[index,'Brawler3Type'] = row['Brawler3Type']+'_2'
df_fixed_map.head(20)



,WinningTeam,BattleTime,playerTag,mode,map,result,StarPlayerTag,StarPlayerBrawler,Brawler1Name,Brawler1Tag,...,Brawler5Name,Brawler5Tag,Brawler6Name,Brawler6Tag,Brawler1Type,Brawler2Type,Brawler3Type,Brawler4Type,Brawler5Type,Brawler6Type
0,1,20200530T122433.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,BROCK,#20C2RRRGY,EMZ,#Y220JJJ9,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER,FIGHTER
1,1,20200530T121808.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,MAX,#22L8CLYQU,PENNY,#28U009JPY,SNIPER,SNIPER_2,SUPPORTER,SUPPORTER,SUPPORTER,SNIPER
2,1,20200530T121207.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,...,BROCK,#9QJP9LP,SPIKE,#98VJCL2PQ,SNIPER,SNIPER_2,SUPPORTER,SUPPORTER,SNIPER,SNIPER
3,1,20200530T120359.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,...,FRANK,#2UCV2R0LV,POCO,#22RURYRV0,SNIPER,SNIPER_2,SUPPORTER,HEAVYWEIGHT,HEAVYWEIGHT,SUPPORTER
4,1,20200530T120028.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,...,BROCK,#98JQVGLP9,JACKY,#8822ULJ9Q,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER,HEAVYWEIGHT
5,1,20200530T115600.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9UR09Q80Y,GENE,GENE,#9UR09Q80Y,...,PIPER,#298R28UG2,BROCK,#QRVCCPPR,SUPPORTER,SNIPER,SNIPER_2,SUPPORTER,SNIPER,SNIPER
6,2,20200530T115304.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PAM,#20JRL29PY,...,GENE,#9UR09Q80Y,BEA,#9U2QJ0P,SUPPORTER,SNIPER,SNIPER_2,SNIPER,SUPPORTER,SNIPER
7,2,20200530T115056.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#20JLGV0QL,...,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,SNIPER,SUPPORTER,SUPPORTER_2,SNIPER,SNIPER,SUPPORTER
8,2,20200530T114747.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9UR09Q80Y,GENE,BEA,#PJJ9P890,...,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,SNIPER,FIGHTER,SUPPORTER,SNIPER,SNIPER,SUPPORTER
11,2,20200530T143201.000Z,#8P80J8RC,brawlBall,Backyard Bowl,-1,#YCC8VGRU,PAM,MR. P,#8P80J8RC,...,PAM,#YCC8VGRU,PIPER,#9JY82J8U,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER,SNIPER


# Find Frequent Combinations of Brawler Types
###### First we want to find the frequent used combinations of brawler types. This is done by computing the frequent itemsets.

In [141]:
combinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team1 = []
    team2 = []
    team1.append(row["Brawler1Type"])
    team1.append(row["Brawler2Type"])
    team1.append(row["Brawler3Type"])
    team2.append(row["Brawler4Type"])
    team2.append(row["Brawler5Type"])
    team2.append(row["Brawler6Type"])

    combinations.append(team1)
    combinations.append(team2)

In [142]:
te = TransactionEncoder()
te_ary = te.fit(combinations).transform(combinations)
df_fixed_mapFreqSetsCombinations = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsCombinations


,ASSASSIN,ASSASSIN_2,ASSASSIN_3,FIGHTER,FIGHTER_2,FIGHTER_3,HEAVYWEIGHT,HEAVYWEIGHT_2,HEAVYWEIGHT_3,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER_2,SUPPORTER_3,THROWER
0,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
1,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False
1802,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False
1803,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
1804,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False


In [143]:
frequent_itemsets_combinations = apriori(df_fixed_mapFreqSetsCombinations, min_support=0.001, use_colnames=True)
frequent_itemsets_combinations['length'] = frequent_itemsets_combinations['itemsets'].apply(lambda x: len(x))

In [144]:
frequent_itemsets_combinations[ (frequent_itemsets_combinations['length'] == 3) ].sort_values(by=["support"], ascending=False)




,support,itemsets,length
87,0.075305,"(FIGHTER, SUPPORTER, SNIPER)",3
75,0.073090,"(SUPPORTER, ASSASSIN, SNIPER)",3
97,0.059801,"(SNIPER_2, SUPPORTER, SNIPER)",3
96,0.052049,"(SNIPER_2, SNIPER_3, SNIPER)",3
93,0.047619,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",3
74,0.035991,"(SNIPER_2, ASSASSIN, SNIPER)",3
67,0.035437,"(FIGHTER, ASSASSIN, SNIPER)",3
98,0.033776,"(SUPPORTER, SNIPER, SUPPORTER_2)",3
71,0.027132,"(HEAVYWEIGHT, ASSASSIN, SNIPER)",3
72,0.024917,"(SUPPORTER, HEAVYWEIGHT, ASSASSIN)",3


# Find Frequent Winning Combinations of Brawler Types
###### We want to find the frequent winning combinations of brawler types. This is done by computing the frequent itemsets.

In [145]:
winningCombinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])
    elif row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])

    if len(team) != 0:
        winningCombinations.append(team)

In [146]:
te = TransactionEncoder()
te_ary = te.fit(winningCombinations).transform(winningCombinations)
df_fixed_mapFreqSetsWinner = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsWinner

,ASSASSIN,ASSASSIN_2,FIGHTER,FIGHTER_2,FIGHTER_3,HEAVYWEIGHT,HEAVYWEIGHT_2,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER_2,SUPPORTER_3,THROWER
0,False,False,False,False,False,False,False,True,True,False,True,False,False,False
1,False,False,False,False,False,False,False,True,True,False,True,False,False,False
2,False,False,False,False,False,False,False,True,True,False,True,False,False,False
3,False,False,False,False,False,False,False,True,True,False,True,False,False,False
4,False,False,False,False,False,False,False,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,False,False,True,False,False,False,False,True,False,False,False,False,False,False
869,False,False,True,False,False,False,False,True,False,False,False,False,False,False
870,False,False,True,True,False,False,False,True,False,False,False,False,False,False
871,False,False,True,True,False,False,False,True,False,False,False,False,False,False


In [147]:
frequent_itemsets_winner = apriori(df_fixed_mapFreqSetsWinner, min_support=0.001, use_colnames=True)
frequent_itemsets_winner['length'] = frequent_itemsets_winner['itemsets'].apply(lambda x: len(x))

In [150]:
frequent_itemsets_winner[ (frequent_itemsets_winner['length'] == 3) ].sort_values(by=["support"], ascending=False)

,support,itemsets,length
88,0.077892,"(SNIPER_2, SNIPER_3, SNIPER)",3
80,0.067583,"(FIGHTER, SUPPORTER, SNIPER)",3
68,0.058419,"(SUPPORTER, ASSASSIN, SNIPER)",3
89,0.058419,"(SNIPER_2, SUPPORTER, SNIPER)",3
86,0.046964,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",3
67,0.036655,"(SNIPER_2, ASSASSIN, SNIPER)",3
90,0.035510,"(SUPPORTER, SNIPER, SUPPORTER_2)",3
63,0.029782,"(FIGHTER, ASSASSIN, SNIPER)",3
66,0.017182,"(SUPPORTER, HEAVYWEIGHT, ASSASSIN)",3
92,0.016037,"(SUPPORTER, SUPPORTER_3, SUPPORTER_2)",3


# Find Frequent Loosing Combinations of Brawler Types
###### We want to find the frequent loosing combinations of brawler types. This is done by computing the frequent itemsets.

In [151]:
loosingCombinations = []
counter = 0
for index, row in df_fixed_map.iterrows():
    team = []
    if row["WinningTeam"] == 1 and row["result"] != 0:
        team.append(row["Brawler4Type"])
        team.append(row["Brawler5Type"])
        team.append(row["Brawler6Type"])
    elif row["result"] != 0:
        team.append(row["Brawler1Type"])
        team.append(row["Brawler2Type"])
        team.append(row["Brawler3Type"])

    if len(team) != 0:
        loosingCombinations.append(team)

In [152]:
te = TransactionEncoder()
te_ary = te.fit(loosingCombinations).transform(loosingCombinations)
df_fixed_mapFreqSetsLooser = pd.DataFrame(te_ary, columns=te.columns_)
df_fixed_mapFreqSetsLooser

,ASSASSIN,ASSASSIN_2,ASSASSIN_3,FIGHTER,FIGHTER_2,FIGHTER_3,HEAVYWEIGHT,HEAVYWEIGHT_2,HEAVYWEIGHT_3,SNIPER,SNIPER_2,SNIPER_3,SUPPORTER,SUPPORTER_2,SUPPORTER_3,THROWER
0,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False
869,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False
870,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False
871,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False


In [153]:
frequent_itemsets_looser = apriori(df_fixed_mapFreqSetsLooser, min_support=0.001, use_colnames=True)
frequent_itemsets_looser['length'] = frequent_itemsets_looser['itemsets'].apply(lambda x: len(x))

In [154]:
frequent_itemsets_looser[ (frequent_itemsets_looser['length'] == 3) ].sort_values(by=["support"], ascending=False)



,support,itemsets,length
83,0.087056,"(SUPPORTER, ASSASSIN, SNIPER)",3
95,0.080183,"(FIGHTER, SUPPORTER, SNIPER)",3
107,0.063001,"(SNIPER_2, SUPPORTER, SNIPER)",3
102,0.049255,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",3
75,0.043528,"(FIGHTER, ASSASSIN, SNIPER)",3
79,0.042383,"(HEAVYWEIGHT, ASSASSIN, SNIPER)",3
76,0.036655,"(FIGHTER, SUPPORTER, ASSASSIN)",3
82,0.036655,"(SNIPER_2, ASSASSIN, SNIPER)",3
74,0.035510,"(FIGHTER, HEAVYWEIGHT, ASSASSIN)",3
109,0.032073,"(SUPPORTER, SNIPER, SUPPORTER_2)",3


# Check Winning Percentage for Combinations
###### We want to analyze how often a combination wins relatively to the usage of this combination. Therefore we compute the relative winning percentage for every used combination

In [155]:
length = len(df_fixed_map)
winning_percentage = []
for index, row in frequent_itemsets_combinations.iterrows():
    for idx, winner in frequent_itemsets_winner.iterrows():
        new_row = []    
        if row['itemsets'] == winner['itemsets']:
            total_number_of_combination = length * 2 * row['support']
            total_number_of_wins = length * winner['support']
            win_percentage = total_number_of_wins / total_number_of_combination
            new_row.append(winner['itemsets'])
            new_row.append(win_percentage)
            new_row.append(total_number_of_combination)
            winning_percentage.append(new_row)
winning_percentage = pd.DataFrame(winning_percentage).rename(columns={0: "itemsets", 1: 'winning percentage', 2: 'times picked'})
winning_percentage = winning_percentage[winning_percentage['itemsets'].map(len) == 3].sort_values(by=['winning percentage'], ascending=False)
winning_percentage
            

,itemsets,winning percentage,times picked
87,"(SNIPER_2, SNIPER_3, SNIPER)",0.748264,94.0
92,"(SUPPORTER, THROWER, SUPPORTER_2)",0.689576,3.0
69,"(ASSASSIN, SNIPER, THROWER)",0.689576,3.0
91,"(SUPPORTER, SUPPORTER_3, SUPPORTER_2)",0.689576,21.0
84,"(SNIPER_2, HEAVYWEIGHT, SNIPER)",0.646478,16.0
73,"(FIGHTER, FIGHTER_2, SNIPER)",0.620619,15.0
89,"(SUPPORTER, SNIPER, SUPPORTER_2)",0.525661,61.0
77,"(FIGHTER, HEAVYWEIGHT, THROWER)",0.517182,4.0
67,"(SNIPER_2, ASSASSIN, SNIPER)",0.509225,65.0
85,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",0.493127,86.0


# Correlation between Winning-Rate and Usage of a Combination
###### First we want to remove outliers

In [156]:
cleaned_winning_percentage = winning_percentage.copy()
def remove_outlier(df_fixed_map_in, col_name):
    q1 = df_fixed_map_in[col_name].quantile(0.25)
    q3 = df_fixed_map_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    print(fence_high)
    print(fence_low)
    df_fixed_map_out = df_fixed_map_in.loc[(df_fixed_map_in[col_name] > fence_low) & (df_fixed_map_in[col_name] < fence_high)]
    return df_fixed_map_out
remove_outlier(cleaned_winning_percentage, 'times picked')

110.0
-58.0


,itemsets,winning percentage,times picked
87,"(SNIPER_2, SNIPER_3, SNIPER)",0.748264,94.0
92,"(SUPPORTER, THROWER, SUPPORTER_2)",0.689576,3.0
69,"(ASSASSIN, SNIPER, THROWER)",0.689576,3.0
91,"(SUPPORTER, SUPPORTER_3, SUPPORTER_2)",0.689576,21.0
84,"(SNIPER_2, HEAVYWEIGHT, SNIPER)",0.646478,16.0
73,"(FIGHTER, FIGHTER_2, SNIPER)",0.620619,15.0
89,"(SUPPORTER, SNIPER, SUPPORTER_2)",0.525661,61.0
77,"(FIGHTER, HEAVYWEIGHT, THROWER)",0.517182,4.0
67,"(SNIPER_2, ASSASSIN, SNIPER)",0.509225,65.0
85,"(HEAVYWEIGHT, SUPPORTER, SNIPER)",0.493127,86.0


###### As we see no entries are removed

In [157]:
print(len(winning_percentage))
print(len(cleaned_winning_percentage))

35
35


###### Next we want to check whether there is a correlation between usage of a combination and the winning rate.

In [1]:
cleaned_winning_percentage['times picked'].corr(cleaned_winning_percentage['winning percentage'])


NameError: name 'cleaned_winning_percentage' is not defined

As wee see the correlation is slightly positive with 0.208. Thus, there is a small correlation between the usage of a combination of brawler types and the winning rate. This indicates that it is more probable to win with a certain combination of brawler types on specific maps. it is also probable that some combinations are used very frequently and there fore team 1 and team 2 use the same combination of brawler types which then lowers our computed correlation.